In [40]:
import pandas as pd
import numpy as np

In [3]:
import tensorflow.nn as nn
import tensorflow.math as tm
from tensorflow.keras.metrics import mean_squared_error as mse

In [41]:
data=pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [43]:
data

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,0
1,45,19,34,3,1.5,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0
3,35,9,100,1,2.7,2,0,0,0
4,35,8,45,4,1.0,2,0,0,1
...,...,...,...,...,...,...,...,...,...
4995,29,3,40,1,1.9,3,0,1,0
4996,30,4,15,4,0.4,1,85,1,0
4997,63,39,24,2,0.3,3,0,0,0
4998,65,40,49,3,0.5,2,0,1,0


In [42]:
data.drop(["ZIP Code","ID","CD Account","Securities Account","Personal Loan"],axis=1,inplace=True)

In [44]:
import random as r

In [45]:
r.seed(2)

# Genetic Alg

In [357]:
class genetic:
    def __init__(self,initial_population:int,sequence:tuple)->None:
        self.sequence = sequence
        self.weights = (0,)
        self.ranking = len(sequence)*(0,)
        self.ini_pop = initial_population
        for i in sequence:
            if not(isinstance(i,input_layer)):
                print(i)
                self.weights += (self._randomised_population(initial_population,i.size,i),)
        
    def _randomised_population(self,num:int,dim,layer)->np.ndarray:
        print("DIM : ",dim)
        rand = np.random.uniform(-1000,1000,size = (dim[0],dim[1],num-1))
        w = np.reshape(layer.weights,(layer.weights.shape[0],layer.weights.shape[1],1))
        return np.concatenate((rand,w),axis=-1)
    
    def crossover(self,cross_prob:float,num:int,parent1:float,parent2:float)->float:
        if(r.random()<cross_prob):
            if parent1>parent2:
                choice=r.uniform(parent2 - 0.5*(abs(parent1-parent2)),parent1 + 0.5*(abs(parent1-parent2)))
                return choice
            else:
                choice=r.uniform(parent1 - 0.5*(abs(parent1-parent2)),parent2 + 0.5*(abs(parent1-parent2)))
                return choice
        else:
            return None
        
    def mutation(self,offspring:float,mutate_prob:float)->float:
        if(r.random()<mutate_prob):
            offset = np.random.normal(0,1,1)
            return offspring + offset
        else:
            return offspring
        
    def fitness_function(self,x,y,layer,kind:str,weight,model,no_layer)->np.ndarray:
        ranks = np.zeros(weight.shape)
        
        for i in range(weight.shape[0]):
            for j in range(weight.shape[1]):
                for k in range(weight.shape[2]):
                    layer.weights[i:j] = weight[i,j,k]
                    result = model.frwd_propog(x,y)
                    if kind == "hidden":
                        print("SHAPE : ",weight.shape)
                        ranks[i,j,k] = (2/len(y))*np.sum((y-result))*np.prod([l.weights[i:j] for l in model.sequence[no_layer:]])*np.prod([m.prev_values[0,j]*(1-m.prev_values[0,j]) for m in model.sequence[no_layer-1:]])
                    elif kind == "output":
                        ranks[i,j,k] = (2/len(y))*np.sum((y-result))*layer.prev_values[0,k]
                ind = np.where(ranks[i,j,:] == ranks[i,j,:].min())[0][0]
                layer.weights[i:j] = weight[i,j,ind]
        return ranks
    def fit(self,x,y,model,epochs = 1):
        print("EPOCHS - 1",self.weights)
        self.ranks = (0,)
        for i,j in zip(model.sequence,range(len(model.sequence))):
            if isinstance(i,input_layer):
                continue
            elif(j == len(model.sequence)-1):
                self.ranks+=(self.fitness_function(x,y,i,"output",self.weights[j],model,j+1),)
            else:
                self.ranks+=(self.fitness_function(x,y,i,"hidden",self.weights[j],model,j+1),)
            for _i,range_i in zip(self.ranks[j],range(len(self.ranks[j]))):
                for _j,range_j in zip(_i,range(len(_i))):
                    parent1,parent2 = np.where((np.argsort(_j)) == 0)[0][0],np.where((np.argsort(_j)) == 1)[0][0]
                    parent1, parent2 = self.weights[j][range_i,range_j,parent1],self.weights[j][range_i,range_j,parent2]
                    child = self.crossover(0.7,1,parent1,parent2)
                    if child is None:
                        continue
                    else:
                        child = self.mutation(child,0.5)
                    replace_ind = np.where((np.argsort(_j)) == self.ini_pop-1)[0][0]
                    #i1 = np.where((self.ranks) == _i)[0][0]
                    #j1 = np.where((_i) == _j)[0][0]
                    self.weights[j][range_i:range_j:replace_ind] = child

# Neural Network Layers

In [358]:
class Dense:
    def __init__(self,size,prev_layer_size,random_weights = False,**kwargs):
        self.size = (size,prev_layer_size)
        self.random_weights  = random_weights
        self.initialize_weights(random_weights)
        
    def pass_values(self,values):
        self.prev_values = np.array(values,ndmin=2)
    def initialize_weights(self,random_weights=False):
        if random_weights == False:
            self.weights = np.zeros((self.size[0],self.size[1]))
            self.size+=(prev_layer_size,)
        else:
            self.weights = np.random.uniform(-1000,1000,size = (self.size[0],self.size[1]))
            
    def _linear_function(self):
        print("Shapes = ",self.prev_values.shape,self.weights.T.shape)
        h = self.prev_values@self.weights.T
        return h
    
    def _activation_function(self,h,func = "sigmoid"):
        if(func == "sigmoid"):
            z = tm.sigmoid(h)
        elif(func == "ReLU"):
            z = nn.relu(h)
        elif (func == "softmax"):
            z = nn.softmax(h)
        return z
    
    def forward(self):
        h = self._linear_function()
        self.output = self._activation_function(h)

In [359]:
class input_layer:
    def __init__(self,input_size):
        self.size = input_size
    def set_values(self,values):
        self.input_values = values

In [360]:
class sequential:
    def __init__(self):
        self.sequence = ()
    def add(self,layers:tuple):
        try:
            self.sequence+=layers
        except:
            self.sequence+=(layers,)
            
    def frwd_propog(self,x,y):
        print("\n Only available loss function is MSE\n")
        final_output = []
        for p in range(x.shape[0]):
            self.sequence[0].set_values(x[p,:,:])
            for i in self.sequence:
                if isinstance(i,input_layer):
                    prev_values = i.input_values
                    prev_layer_size = i.size
                else:
                    i.pass_values(prev_values)
                    i.forward()
                    prev_values = i.output
                    prev_layer_size = prev_values.shape[1]
            final_output.append(prev_values)
        final_output = np.reshape(np.array(final_output),(x.shape[0],y.shape[1]))
        self.loss_value = mse(y,final_output)
        return final_output
        
    def train(self,x,y,epochs = 1):
        Gene = genetic(5,self.sequence)
        Gene.fit(x,y,self)
    def predict(x):
        self.sequence[0].set_values(x)
        self.final_output = self.frwd_propog()
        return {"result" : self.final_output, "Loss" : self.loss_value}
    
    def clear(self):
        self.sequence = ()

In [361]:
seq = sequential()
seq.add(input_layer(2))

In [362]:
seq.add(Dense(10,2,True))

In [363]:
seq.add(Dense(1,10,True))

In [364]:
x = np.random.uniform(-100,100,size=(100,1,2))

In [365]:
y = np.random.uniform(-100,100,size = (100,1,1))

In [366]:
seq.train(x,y)

DIM :  (10, 2)
DIM :  (1, 10)
EPOCHS - 1 (0, array([[[-967.8477955 , -295.9477201 ,  565.66496586, -379.62410625,
         -592.65707067],
        [ 623.13266717, -345.00409657, -248.79728775,  354.59598904,
          429.37084252]],

       [[-126.23327652, -596.46726394,  -47.07573334,   -5.13102352,
          637.51074128],
        [ 788.20482745, -885.86255426, -561.46581215, -937.83691956,
         -596.58571932]],

       [[-127.34754588,   -8.05236592, -549.83962867, -271.79680741,
         -657.10865008],
        [-200.90443315, -864.83802899, -918.98722779,  -50.36721487,
         -570.07741027]],

       [[ 215.32808291,  846.2063134 ,  431.36756325, -182.33713426,
          252.54137838],
        [ 340.98410417, -148.18481106,  147.83102373,  478.52058621,
          906.29930668]],

       [[ 930.45606018, -529.48555435, -986.23126367,  366.90539687,
         -839.89530915],
        [ 426.94776962, -103.80314631,  113.41114815, -184.33037521,
          119.06166677]],

     

Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
SHAPE :  (10, 2, 5)

 Only available loss function is MSE

Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10

Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
SHAPE :  (10, 2, 5)

 Only available loss function is MSE

Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 

Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shape

Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes

...

Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes

Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shape

Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes

Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shape

Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes =  (1, 10) (10, 1)
Shapes =  (1, 2) (2, 10)
Shapes

3